In [1]:
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow import keras
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, mean_absolute_error, mean_absolute_percentage_error
import time
from sklearn.preprocessing import MinMaxScaler
import sklearn.metrics
from keras import initializers
import keras_tuner as kt
from tensorflow.keras.optimizers import Adam

def read_csv(file):
    dataframe = pd.read_csv(file=file)
    return dataframe

def build_model(hp):
    model = keras.Sequential()

    model.add(keras.layers.Dense(12, activation="tanh"))

    for i in range(hp.Int("num_layers", 5, 15)):
        model.add(
            keras.layers.Dense(
                units=hp.Int("units_" + str(i), min_value=20, max_value=80, step=10),
                activation="tanh"
            )
        )

    model.add(keras.layers.Dense(1, activation="tanh"))

    optimizer = Adam(
    learning_rate=hp.Float('learning_rate', min_value=0.00001, max_value=0.001, sampling='log')
    )

    model.compile(loss="mean_absolute_error",
                  optimizer=optimizer,
                  metrics=["mean_absolute_error","mean_squared_error"])
    
    return model

Using TensorFlow backend


In [2]:
# Cargar el archivo CSV con punto y coma como delimitador
data = pd.read_csv('Cluster0ReadyToNN.csv', delimiter=';')

# Crear un diccionario para almacenar los objetos scaler por grupo
scalers = {}

# Iterar sobre los grupos únicos en Column15
for group in data['Column15'].unique():
    # Filtrar datos por grupo
    group_data = data[data['Column15'] == group]

    # Seleccionar las columnas para normalización (las 13 primeras)
    features = group_data.iloc[:, :13]

    # Normalizar los datos con MinMaxScaler
    scaler = MinMaxScaler()
    normalized_data = scaler.fit_transform(features)

    # Almacenar el scaler en el diccionario
    scalers[group] = scaler

    # Actualizar el DataFrame con los datos normalizados
    data.loc[data['Column15'] == group, 'Column1':'Column13'] = normalized_data

In [3]:
# Dividir los datos en entrenamiento (70%) y temporal (30%)
train_temp_data, test_data = train_test_split(data, test_size=0.3, stratify=data['Column15'], random_state=0)

# Dividir el temporal en entrenamiento (70%) y validación (30%)
train_data, validation_data = train_test_split(train_temp_data, test_size=0.3, stratify=train_temp_data['Column15'], random_state=0)

# Separar características (X) y columna objetivo (y) para entrenamiento
X_train_norm = train_data.iloc[:, :12]
y_train_norm = train_data['Column13']

# Separar características (X) y columna objetivo (y) para validación
X_val_norm = validation_data.iloc[:, :12]
y_val_norm = validation_data['Column13']

# Separar características (X) y columna objetivo (y) para prueba
X_test_norm = test_data.iloc[:, :12]
y_test_norm = test_data['Column13']

In [7]:
#Params
epochs_val = 100
batch_size_val = 256
max_trials_val = 25

In [8]:
start_time = time.time()
tuner = kt.RandomSearch(
    build_model,
    objective="val_loss",
    max_trials=max_trials_val,
    directory='my_tuner_dir',
    project_name='DFFNN_optimization_norm'
)
callback = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=10, verbose=1, min_delta=1E-4)
tuner.search(X_train_norm, y_train_norm, epochs=epochs_val, batch_size=batch_size_val, callbacks=[callback], validation_data=(X_val_norm, y_val_norm))
tuner.results_summary()
end_time = time.time()

best_model = tuner.get_best_models()[0]
print("Best Model: ", best_model)


history = best_model.fit(X_train_norm, y_train_norm, epochs=epochs_val, batch_size=batch_size_val, validation_data=(X_val_norm, y_val_norm), callbacks=[callback])

y_pred = best_model.predict(X_test_norm)
#print(predictions)

Trial 25 Complete [00h 02m 03s]
val_loss: 0.08507491648197174

Best val_loss So Far: 0.0842648521065712
Total elapsed time: 00h 33m 28s
Results summary
Results in my_tuner_dir\DFFNN_optimization_norm
Showing 10 best trials
Objective(name="val_loss", direction="min")

Trial 09 summary
Hyperparameters:
num_layers: 13
units_0: 20
units_1: 50
units_2: 40
units_3: 70
units_4: 20
learning_rate: 0.0009226604447116868
units_5: 20
units_6: 50
units_7: 20
units_8: 20
units_9: 60
units_10: 40
units_11: 80
units_12: 30
units_13: 30
units_14: 60
Score: 0.0842648521065712

Trial 22 summary
Hyperparameters:
num_layers: 8
units_0: 80
units_1: 80
units_2: 20
units_3: 30
units_4: 40
learning_rate: 0.0006857926194454209
units_5: 40
units_6: 70
units_7: 40
units_8: 20
units_9: 60
units_10: 80
units_11: 30
units_12: 40
units_13: 50
units_14: 70
Score: 0.08449201285839081

Trial 12 summary
Hyperparameters:
num_layers: 6
units_0: 20
units_1: 50
units_2: 50
units_3: 50
units_4: 70
learning_rate: 0.00028704953

In [11]:
# Crear un DataFrame con las predicciones desnormalizadas y los valores reales
resultados = pd.DataFrame({'Valor Real': y_test_norm.values.flatten(), 'Predicciones': y_pred.flatten()})
print(resultados)

# Agregar la columna de predicciones al conjunto de prueba
test_data['Predicted_Column13'] = y_pred.flatten()

# Crear un DataFrame para almacenar los resultados desnormalizados
desnormalized_test_data = test_data.copy()

# Desnormalizar 'Column1' a 'Column13' y 'Predicted_Column13' según la normalización por grupos
for group, scalerY in scalers.items():
    # Filtrar el conjunto de prueba correspondiente al grupo
    group_test_data = test_data[test_data['Column15'] == group]

    # Seleccionar las columnas normalizadas para desnormalizar
    normalized_features = group_test_data[['Column1', 'Column2', 'Column3', 'Column4', 'Column5', 'Column6', 'Column7', 'Column8', 'Column9', 'Column10', 'Column11', 'Column13', 'Predicted_Column13']]

    # Desnormalizar los datos utilizando el objeto scalerY correspondiente
    original_data = scalerY.inverse_transform(normalized_features)

    # Crear un DataFrame temporal para almacenar los datos desnormalizados
    temp_df = pd.DataFrame(original_data, columns=['Column1', 'Column2', 'Column3', 'Column4', 'Column5', 'Column6', 'Column7', 'Column8', 'Column9', 'Column10', 'Column11', 'Column13', 'Predicted_Column13'])

    # Actualizar el DataFrame desnormalizado con los datos desnormalizados
    desnormalized_test_data.loc[desnormalized_test_data['Column15'] == group, ['Column1', 'Column2', 'Column3', 'Column4', 'Column5', 'Column6', 'Column7', 'Column8', 'Column9', 'Column10', 'Column11', 'Column13', 'Predicted_Column13']] = temp_df.values

# Imprimir el conjunto de prueba después de la desnormalización
print(desnormalized_test_data)

# Eliminar todas las columnas excepto las últimas cuatro
resultados = desnormalized_test_data.iloc[:, -4:]

# Imprimir el conjunto de prueba después de la eliminación de columnas
print(resultados)

       Valor Real  Predicciones
0        0.128378      0.200324
1        0.347826      0.279113
2        0.051724      0.114578
3        0.295400      0.244977
4        0.155556      0.098535
...           ...           ...
51171    0.321429      0.132531
51172    0.011936      0.031590
51173    0.053030      0.104980
51174    0.312155      0.252653
51175    0.600000      0.462336

[51176 rows x 2 columns]


C:\Users\AdrianGilGamboa\AppData\Local\Temp\ipykernel_17112\1696851233.py:26: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '[19.51019157 21.06286038 22.83313935 19.39220949 17.75914561 18.1616634
 20.35502601 18.84101207 38.22458571 20.27732049 45.46156937 19.96335268
 46.14192778 17.29043203 17.91836925 19.87818758 21.83925015]' has dtype incompatible with float32, please explicitly cast to a compatible dtype first.
  desnormalized_test_data.loc[desnormalized_test_data['Column15'] == group, ['Column1', 'Column2', 'Column3', 'Column4', 'Column5', 'Column6', 'Column7', 'Column8', 'Column9', 'Column10', 'Column11', 'Column13', 'Predicted_Column13']] = temp_df.values


        Column1  Column2  Column3  Column4  Column5  Column6  Column7  \
24905      58.0     66.0     62.0     56.0     77.0     60.0    170.0   
155320    101.0     39.0     24.0     28.0     65.0     32.0     28.0   
8745       30.0     30.0     29.0     31.0     31.0     31.0     29.0   
60462     193.0     59.0     48.0     87.0    186.0     59.0     47.0   
4241       35.0     29.0     37.0     31.0     29.0     32.0     32.0   
...         ...      ...      ...      ...      ...      ...      ...   
132964     37.0     35.0     33.0     35.0     34.0     36.0     21.0   
130033      5.0      8.0      7.0     11.0      1.0      7.0      8.0   
124375    137.0    120.0     63.0     60.0     54.0     73.0      6.0   
50855     185.0     32.0     49.0     67.0     51.0     37.0     56.0   
5442       29.0     65.0     44.0     31.0     50.0     69.0     48.0   

        Column8  Column9  Column10  Column11  Column12  Column13 Column14  \
24905      69.0     66.0      48.0      53.0  

In [12]:
# Obtener y_test_norm de la primera columna de resultados
y_test_norm = resultados['Column13']

# Obtener y_pred de la última columna del conjunto de prueba después de la desnormalización
y_pred = desnormalized_test_data['Predicted_Column13']

# Imprimir y_test_norm y y_pred
print("y_test_norm:", y_test_norm)
print("y_pred:", y_pred)


# Imprimir el DataFrame
print(resultados)


# Calcular RMSE con datos desnormalizados
rmse = np.sqrt(mean_squared_error(y_test_norm, y_pred))
print(f'RMSE en el conjunto de prueba: {rmse}')

# Calcular MAE con datos desnormalizados
mae = mean_absolute_error(y_test_norm, y_pred)
print(f'MAE en el conjunto de prueba: {mae}')

#Calcular MAPE con datos desnormalizados
print("Mean absolute percentage error (MAPE): %f" % mean_absolute_percentage_error(y_test_norm, y_pred))

# Calcula la duración del entrenamiento en segundos
training_duration = end_time - start_time
# Imprime el tiempo de entrenamiento en segundos y en formato de horas, minutos y segundos
print(f'Tiempo de busqueda de parametros: {training_duration:.2f} segundos')
print(f'Tiempo de busqueda de parametros (HH:MM:SS): {int(training_duration // 3600)}:{int((training_duration % 3600) // 60)}:{int(training_duration % 60)}')

y_test_norm: 24905      41.0
155320     59.0
8745       23.0
60462     150.0
4241       34.0
          ...  
132964     65.5
130033     10.0
124375     13.0
50855     145.0
5442       85.0
Name: Column13, Length: 51176, dtype: float64
y_pred: 24905      51.647930
155320     47.937202
8745       26.645502
60462     129.175389
4241       26.302183
             ...    
132964     19.710875
130033     24.818666
124375     19.857371
50855     123.460560
5442       69.856937
Name: Predicted_Column13, Length: 51176, dtype: float64
        Column13 Column14  Column15  Predicted_Column13
24905       41.0   2005/2      8751           51.647930
155320      59.0   2006/3     56704           47.937202
8745        23.0   2015/2      3059           26.645502
60462      150.0   2011/3     20496          129.175389
4241        34.0   2009/4      1242           26.302183
...          ...      ...       ...                 ...
132964      65.5   2006/3     46569           19.710875
130033      10.0   200

In [ ]:
#MAPE by year calculated in KNIME